# NOTEBOOK DEPLOYEMENT CLOUD D'UNE APPLICATION DE MISE EN OEUVRE DES CLUSTERS DE CALCUL PYSPARK( Jobs Spark)

# 1 Introduction;

L'objet de ce projet est de deployer une application Spark jobs dans le cloud sur une plateforme Amazon Web Service.
La conception de ce projet necessite de suivre la démarche analytique suivante:

# a_Etablir une  surveillance du temps de calcul sur l'environnement local en vue d'une comparaison avec les performances en temps de l'environnement cloud.

# b_Urbaniser la localisation  des données d'entrée et de sortie sur les differents entrepots de données sur l'environnement local
    

# c_Charger les données
      Explorez Spark Dataframe au format 'image'
      Nous garderons l'origine pour obtenir le chemin et extraire l'étiquette
      Autre façon de traiter l'image
    
# d_Utiliser un CNN comme extracteur de fonctionnalités

###   ===>   Faire une caractérisation  des images :
        Définir la logique de chargement et de caractéristiques des images dans une UDF Pandas
    
###  ===> Procéder à une réduction dimensionnelle
        Transformer des tableaux en vecteurs pour effectuer une réduction
        Initialiser et appliquer PCA
        Transformation inverse : des vecteurs au tableau - c'est-à-dire la lisibilité des pandas
            
# c_Collecter et Sauvegarder les résultats en format parquet en vue d'une exploitation ultérieure.
          


# 2 Importation des librairies

Dans ce cahier, nous gérons via pySpark, sql & ml, la caractérisation et la réduction de dimension d'une collection d'images.
L'objectif est de construire une architecture évolutive pour effectuer les transformations attendues et permettre une croissance rapide de la collection d'images en gardant un temps de calcul viable.

In [1]:
# packages standard
import pandas as pd
import numpy as np
import time
import os
import io
############################
import sys
import pyspark
from platform import python_version
############################
#Packlages Pyspark
############################
from PIL import Image,ImageDraw
from typing import Iterator
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession,Row
from PIL import Image, ImageDraw
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split,udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BinaryType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
import pyarrow
#transform
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
##################################
#Packages Keras
##################################
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras import Model
import tensorflow as tf


In [2]:
#Selection de l'environnement de deploiement
Environnement_Test_Local=True
    
#===================================================================================================================    
if Environnement_Test_Local: 
    import findspark
    findspark.init()
    # 4 Configuration des variables utilisateurs sur l'environnement de deploiement de Test après chargement de Python,Java et Spark dans le serveur windows 

    os.environ['PYSPARK_PYTHON'] = sys.executable
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable          

    ## Urbanisation en local

    PATH='C:\\Users\\dgama\\Desktop\\OPENCLASSROOMS_INFO\\REPERTOIRE_P8_BIS'

    PATH_R='C:\\Users\\dgama\\Desktop\\OPENCLASSROOMS_INFO\\REPERTOIRE_P8_TEMP'

    PATH_Data = PATH+'\Test2'
    PATH_Result = PATH_R+'\Results'
    print("********Bienvenue sur votre environnement de Test local pour jobs spark***********"+\
         "\n\nLa version Python extrait de l environnement Conda pyspark_env est:"\
         ,sys.version +"\nLa version de l'application Spark est:"\
         ,pyspark.__version__ +"\nL'OS de l'environnement de deploiement est:"\
         ,sys.platform +"\n\nPATH:        "+\
         PATH+"\nPATH_Data:   "+\
         PATH_Data +"\nPATH_Result:"+ PATH_Result)
#==========================================================================================================================       
else:
   
    # 4 Configuration des variables utilisateurs sur l'environnement de deploiement de Test après chargement de Python,Java et Spark dans le serveur windows 

    ## Urbanisation en local
    PATH='s3://myprojet8bucket'

    PATH_DATA='s3://myprojet8bucket/Input-Test/*'

    PATH_FEATURES='s3://myprojet8bucket/Output_Data/features/'

    PATH_FINAL_FEATURES='s3://myprojet8bucket/Output_Data/final_features/'

    PATH ='s3://myprojet8bucket'
    PATH_Data = PATH+'/Input-Test/*'
    PATH_FEATURES = PATH+'/Output_Data/features'
    PATH_FINAL_FEATURES=PATH+'/Output_Data/final_features'
    print("********Bienvenue sur votre environnement de production Amazon Web Service pour jobs spark***********"+\
         "\n\nLa version Python extrait de l environnement Aws_Pyspark est:"\
         ,sys.version +"\nLa version de l'application Spark est:"\
         ,pyspark.__version__ +"\nL'OS de l'environnement de deploiement est:"\
         ,sys.platform +"\n\nPATH:        "+\
          PATH+"\nPATH_DATA:   "+\
          PATH_Data+"\nPATH_FEATURES: "\
          +PATH_FEATURES +"\nPATH_FINAL_FEATURES: "\
           +PATH_FINAL_FEATURES)
#================================

********Bienvenue sur votre environnement de Test local pour jobs spark***********

La version Python extrait de l environnement Conda pyspark_env est: 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC v.1929 64 bit (AMD64)]
La version de l'application Spark est: 3.3.2
L'OS de l'environnement de deploiement est: win32

PATH:        C:\Users\dgama\Desktop\OPENCLASSROOMS_INFO\REPERTOIRE_P8_BIS
PATH_Data:   C:\Users\dgama\Desktop\OPENCLASSROOMS_INFO\REPERTOIRE_P8_BIS\Test2
PATH_Result:C:\Users\dgama\Desktop\OPENCLASSROOMS_INFO\REPERTOIRE_P8_TEMP\Results


In [3]:
# 7 Session Spark
if Environnement_Test_Local: 
        spark = (SparkSession
                     .builder
                     .appName('P8')
                     .master('local')
                     .config("spark.sql.parquet.writeLegacyFormat", 'true')
                     .getOrCreate()
                )
        # check wether arrow should be enabled by this setting
        spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
        
        # open spark UI for app monitoring
        
        
        sc = spark.sparkContext

        
        
else:
       if not 'sc' in globals():
            sc=pyspark.SparkContext()       #pour démarrer
            # check wether arrow should be enabled by this setting
            spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
        
            


In [4]:
spark

In [5]:
sc = spark.sparkContext

In [6]:
# function to load data into into a spark_d
df_img = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [7]:

# number of images in this sample
df_img.count()

295

In [8]:
df_img.show(5)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/C:/Users/dg...|2023-02-22 00:15:...|  9353|[FF D8 FF E0 00 1...|
|file:/C:/Users/dg...|2023-02-22 00:15:...|  9330|[FF D8 FF E0 00 1...|
|file:/C:/Users/dg...|2023-02-22 00:15:...|  9279|[FF D8 FF E0 00 1...|
|file:/C:/Users/dg...|2023-02-22 00:15:...|  9274|[FF D8 FF E0 00 1...|
|file:/C:/Users/dg...|2023-02-22 00:15:...|  9216|[FF D8 FF E0 00 1...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows



In [9]:
### Path de l'image conservé et nous ajoutons une colonne contenant les labels de chaque image

df_img = df_img.withColumn('label', element_at(split(df_img['path'], '/'),-2))
print(df_img.printSchema())
print(df_img.select('path','label').show(5,False))


root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------------------------------------------------+--------------+
|path                                                                                              |label         |
+--------------------------------------------------------------------------------------------------+--------------+
|file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_INFO/REPERTOIRE_P8_BIS/Test2/apple_golden_1/r0_111.jpg|apple_golden_1|
|file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_INFO/REPERTOIRE_P8_BIS/Test2/apple_golden_1/r0_107.jpg|apple_golden_1|
|file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_INFO/REPERTOIRE_P8_BIS/Test2/apple_golden_1/r0_115.jpg|apple_golden_1|
|file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_INFO/REPERTOIRE_P8_BIS/Test2/apple_golden_1/r0_11

In [10]:
# 8 Préparation du Modèle

model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224,224,3))

new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)



In [11]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [12]:
# get weights as broadcasted variable over nodes (provide a copy to each node)

brodcast_weights = sc.broadcast(new_model.get_weights())

In [13]:
# 9 Caractérisation (Featurizer)  et Reduction dimensionnelle(Reducer)
'''
Nous utilisons le transfer learning pour featuriser les images .
Nous allons procéder à la réduction des la dimension des torseurs d'images en transformons les matrices d'images
en Vecteurs d'images'''

start = time.perf_counter()
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)


C:\Users\dgama\anaconda3\envs\pyspark_env\lib\site-packages\pyspark\sql\pandas\functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [15]:
featurized_df = df_img.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

featurized_df.show()

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|file:/C:/Users/dg...|      apple_golden_1|[0.0, 0.38403308,...|
|file:/C:/Users/dg...|         apple_red_1|[0.63280374, 0.04...|
|file:/C:/Users/dg...|              pear_1|[0.4929007, 0.0, ...|
|file:/C:/Users/dg...|              pear_1|[0.3754005, 0.0, ...|
|file:/C:/Users/dg...|            carrot_1|[0.115660794, 0.3...|
|file:/C:/Users/dg...|  apple_red_yellow_1|[0.059212092, 0.2...|
|file:/C:/Users/dg...|              pear_3|[0.05573111, 0.09...|
|file:/C:/Users/dg...|    apple_braeburn_1|[0.49349692, 0.88...|
|file:/C:/Users/dg...|          zucchini_1|[0.07602775, 0.64...|
|file:/C:/Users/dg...|   apple_pink_lady_1|[0.23992004, 0.92...|
|file:/C:/Users/dg...|apple_granny_smith_1|[0.39774752, 1.36...|
|file:/C:/Users/dg...|apple_red_delicios_1|[0.2192569, 0.143...|
|file:/C:/Users/dg...|   

In [16]:
featurized_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [17]:
if Environnement_Test_Local:
    featurized_df.write.mode("overwrite").parquet(PATH_Result)

    featurized_dt = pd.read_parquet(PATH_Result, engine='pyarrow')

    featurized_dt.head()
else:
    featurized_df.write.mode("overwrite").parquet(PATH_FEATURES)

    featurized_dt = pd.read_parquet(PATH_FEATURES, engine='pyarrow')

    featurized_dt.head()
    
    

In [18]:
# get the number of paritions

print(featurized_df.rdd.getNumPartitions())

### On valide que les vecteur des caracterisque des images sint  bien de dimension 1280
featurized_dt.loc[0,'features'].shape

20


(1280,)

In [19]:
# from Array to Vectors for PCA

array_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

vectorized_df = featurized_df.withColumn('cnn_vectors', array_to_vector_udf('features'))

vectorized_df.show(5, True)

+--------------------+--------------+--------------------+--------------------+
|                path|         label|            features|         cnn_vectors|
+--------------------+--------------+--------------------+--------------------+
|file:/C:/Users/dg...|apple_golden_1|[0.0, 0.38403308,...|[0.0,0.3840330839...|
|file:/C:/Users/dg...|   apple_red_1|[0.63280374, 0.04...|[0.63280373811721...|
|file:/C:/Users/dg...|        pear_1|[0.4929007, 0.0, ...|[0.49290069937705...|
|file:/C:/Users/dg...|        pear_1|[0.3754005, 0.0, ...|[0.37540051341056...|
|file:/C:/Users/dg...|      carrot_1|[0.115660794, 0.3...|[0.11566079407930...|
+--------------------+--------------+--------------------+--------------------+
only showing top 5 rows



In [20]:
# 10 Initialisation et Application PCA

# Results may vary with larger dataset, creates an heavy task Action that would affect overall performance.

# reduce with PCA - set k Max to determine the adequate nb of principal components

pca = PCA(k=20, inputCol='cnn_vectors', outputCol='pca_vectors')
model = pca.fit(vectorized_df)

# apply pca reduction

reduced_df = model.transform(vectorized_df)

reduced_df.show(5, True)


+--------------------+--------------+--------------------+--------------------+--------------------+
|                path|         label|            features|         cnn_vectors|         pca_vectors|
+--------------------+--------------+--------------------+--------------------+--------------------+
|file:/C:/Users/dg...|apple_golden_1|[0.0, 0.38403308,...|[0.0,0.3840330839...|[5.45436830699842...|
|file:/C:/Users/dg...|   apple_red_1|[0.63280374, 0.04...|[0.63280373811721...|[-8.5448584676211...|
|file:/C:/Users/dg...|        pear_1|[0.4929007, 0.0, ...|[0.49290069937705...|[4.42213854561953...|
|file:/C:/Users/dg...|        pear_1|[0.3754005, 0.0, ...|[0.37540051341056...|[4.63591629411558...|
|file:/C:/Users/dg...|      carrot_1|[0.115660794, 0.3...|[0.11566079407930...|[1.95431186523843...|
+--------------------+--------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [21]:
# 11 Transformation inverse: Des vecteurs vers un tableau - i.e. DataFrame Pandas

# from Array to Vectors for PCA
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

final_df = reduced_df.withColumn('features', vector_to_array_udf('pca_vectors'))

final_df.show(5, True)


+--------------------+--------------+--------------------+--------------------+--------------------+
|                path|         label|            features|         cnn_vectors|         pca_vectors|
+--------------------+--------------+--------------------+--------------------+--------------------+
|file:/C:/Users/dg...|apple_golden_1|[5.454368, 1.8741...|[0.0,0.3840330839...|[5.45436830699842...|
|file:/C:/Users/dg...|   apple_red_1|[-8.544859, 2.057...|[0.63280373811721...|[-8.5448584676211...|
|file:/C:/Users/dg...|        pear_1|[4.4221387, -7.90...|[0.49290069937705...|[4.42213854561953...|
|file:/C:/Users/dg...|        pear_1|[4.635916, -7.931...|[0.37540051341056...|[4.63591629411558...|
|file:/C:/Users/dg...|      carrot_1|[1.9543118, 6.150...|[0.11566079407930...|[1.95431186523843...|
+--------------------+--------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [24]:
# 12 Collectes et Sauvegardes finales des Jobs Spark

# write local results on parquet file
if Environnement_Test_Local:
    final_df.write.mode('overwrite').parquet(PATH_Result)

    # read local results from parquet file

    pd_final_df = pd.read_parquet(PATH_Result, engine='pyarrow')
    
else:
     final_df.write.mode('overwrite').parquet(PATH_FINAL_FEATURES)

    # read local results from parquet file

     pd_final_df = pd.read_parquet(PATH_FINAL_FEATURES, engine='pyarrow')
    

# size of the results df

pd_final_df.info(verbose=False, memory_usage="deep")

# overview
pd_final_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Columns: 5 entries, path to pca_vectors
dtypes: object(5)
memory usage: 228.5 KB


,path,label,features,cnn_vectors,pca_vectors
0,file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_IN...,apple_golden_1,"[5.454368, 1.8741764, 6.035341, 0.610102, -3.9...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
1,file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_IN...,apple_red_1,"[-8.544859, 2.0570605, 1.6779149, 2.8956733, -...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
2,file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_IN...,pear_1,"[4.4221387, -7.907045, 5.021093, -4.917258, 4....","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
3,file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_IN...,pear_1,"[4.635916, -7.9319572, 5.2284975, -5.740709, 4...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."
4,file:/C:/Users/dgama/Desktop/OPENCLASSROOMS_IN...,carrot_1,"[1.9543118, 6.150467, -8.640552, 1.3370864, -9...","{'type': 1, 'size': None, 'indices': None, 'va...","{'type': 1, 'size': None, 'indices': None, 'va..."


In [25]:
spark.stop()